In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /var/tmp/pip-req-build-tblz7ahb
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /var/tmp/pip-req-build-tblz7ahb
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [ ]:
from collections import Counter
import pandas as pd
import clip
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Loading the dataset
articles = pd.read_csv('articles.csv')
#print(articles.iloc[0])
subset = articles

'''
# drop items that have the same description
subset = articles.drop_duplicates("detail_desc").copy()

# remove items of unkown category
subset = subset[~subset["product_group_name"].isin(["Unknown"])]
'''

def longer_than(x, max_length):

  try:
    LEN = 500
    # use clip tokenizer with a very large value to be sure
    tokens = clip.tokenize(x, context_length=LEN).cpu().numpy()
    # empty positions are tagged with 0s e.g. [1, 4, 2, 6, 78, 3, 0, 0, 0, 0, 0, 0, 0...]
    # minimum with [1, 1, 1, 1, 1...] gives you [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0...]
    # summing this gives you the length
    length = np.minimum(tokens, np.ones(LEN)).sum()
  except:
    return False
  return length <= max_length


# Drop items that have a description longer than 77 tokens
subset = subset[subset["detail_desc"].apply(lambda x : longer_than(x, 77))]

# We also drop products types that do not occur very frequently in this subset of data
most_frequent_product_types = [k for k, v in dict(Counter(subset["product_type_name"].tolist())).items() if v > 10]
subset = subset[subset["product_type_name"].isin(most_frequent_product_types)]

In [ ]:
# Create train / test subsets
train_subset, test_subset = train_test_split(subset, test_size=0.2, random_state=42)

# Save them
train_subset.to_csv("train_subset.csv", index=False)
test_subset.to_csv("test_subset.csv", index=False)

In [ ]:
# Create a subset without duplicate detail_desc
subset_no_duplicates = subset.drop_duplicates("detail_desc").copy()

# Drop duplicate detail_desc (i.e. same item but often different color etc.)
train_subset_no_duplicates, test_subset_no_duplicates = train_test_split(subset_no_duplicates, test_size=0.2, random_state=42)

# Save them
train_subset_no_duplicates.to_csv("train_subset_no_duplicates.csv", index=False)
test_subset_no_duplicates.to_csv("test_subset_no_duplicates.csv", index=False)

In [ ]:
print(len(train_subset))
print(len(test_subset))

print(len(train_subset_no_duplicates))
print(len(test_subset_no_duplicates))

82793
20699
33969
8493
